In [25]:

with open("../backend/logging/search.log", "r") as f, open("pre_file.csv", "w") as f2:
    query_actual = None

    line_wait = []

    for line_ in f.readlines():
        
        time, event, query, documents_ = line_.split(";")
        time, event, query, documents_ = time, event.strip(), query.strip(), documents_.strip()
        
        if not query_actual:
            query_actual = query
            line_wait = [line_,]
            continue

        if query_actual == query:
            line_wait.append(line_)
        else:
            len_be = len(line_wait)
            list_be = line_wait.copy()
            time_local, event_local, query_local, documents_local = (line_wait.pop(0)).split(";")
            time_local, event_local, query_local, documents_local = time_local, event_local.strip(), query_local.strip(), documents_local.strip()

            documents_local_rank = documents_local.split("|")

            if len(line_wait) > 0:
                test_event = (line_wait[0].split(";")[1]).strip()
                while test_event == "Search.viewed":
                    time_local, event_local, query_local, documents_local = (line_wait.pop(0)).split(";")
                    time_local, event_local, query_local, documents_local = time_local, event_local.strip(), query_local.strip(), documents_local.strip()

                    documents_local_rank = documents_local.split("|")

                    if len(line_wait) == 0: 
                        break
                    
                    test_event = (line_wait[0].split(";")[1]).strip()
           
            try:
                if len(line_wait) > 0:

                    last = (line_wait[-1].split(";")[3]).strip()
                    position = documents_local_rank.index(last)

                    put_file_documents_local_rank = documents_local_rank[:position+1]

                    for doc in put_file_documents_local_rank:
                        f2.write(f"{query_local},{doc},1,0\n")
                        
            except Exception as e:
                raise ValueError(e)
            

            for line_w in line_wait:
                doc = (line_wait[-1].split(";")[3]).strip()
                f2.write(f"{query_local},{doc},0,1\n")


            query_actual = query
            line_wait = [line_,]
    

In [26]:
!sort -t ',' -k 1n pre_file.csv > pre_file_sorted.csv

In [ ]:
!rm pre_file.csv

In [27]:
now_q = ""
now_d = ""

v_c = (0,0)

first = True

with open("pre_file_sorted.csv", "r") as f3, open("features_file.csv", "w") as f4:
    for line_ in f3.readlines():
        try:
            list_all = [x.strip() for x in line_.split(",")]

            click, view, document = list_all.pop(),list_all.pop(),list_all.pop()
            query = ",".join(list_all)

            if first:
                now_q = query
                now_d = document
                v_c = (int(view),int(click))
                first = False

            if (now_q, now_d) != (query,document):
                f4.write(f"{now_q},{now_d},{v_c[0]},{v_c[1]}\n")
                now_q = query
                now_d = document
                v_c = (int(view),int(click))
            else:
                v_c = (v_c[0] + int(view), v_c[1] + int(click))
        except:
            print("error", line_)

In [28]:
import numpy as np
with open("features_file.csv", "r") as f5, open("reference_colletion.csv", "w") as f6:

    f6.write(f"query,document,label\n")

    for line_ in f5.readlines():

        list_all = [x.strip() for x in line_.split(",")]

        click, view, document = list_all.pop(),list_all.pop(),list_all.pop()
        query = " ".join(list_all)

        view, click = int(view), int(click)
        prior_a = 5
        prior_b = 25
        posterior_a = click+prior_a
        posterior_b = prior_b+(view-click)
        label = np.round(((posterior_a  / (posterior_b+posterior_a))*32)).astype("int")
        f6.write(f"{query},{document},{label}\n")